In [24]:
import os
import sys
import json
import random
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import keras
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [119]:
def load_json(json_path, artists=[]):
    if (os.path.isfile(json_path)):
        print("json")
        with open(json_path) as f:
            song_data = json.load(f)
            return song_data['songs']
        
    elif (os.path.isdir(json_path)):
        data = []
        json_files = []
        if (len(artists) > 0):
            for artist in artists:
                json_files = json_files + [json_file for json_file in os.listdir(json_path) if ((json_file.endswith('.json')) & (artist in json_file))]
        else:
            json_files = [json_file for json_file in os.listdir(json_path) if json_file.endswith('.json')]

        for json_file in json_files:
            path_to_json = os.path.join(json_path, json_file)
            with open(path_to_json) as f:
                song_data = json.load(f)
                data = data + song_data['songs']
        
        return data
    
    
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)


In [120]:
# Parameterts
maxlen = 60  # extraxt sequences of n characters
step = 3     # sample new seq every n characters
json_path = '../data/deutsch'
artists = ['Cro']

## Datapreprocessing

In [154]:
#######
#  CHANGE - added filtering 
#  Load and filter lyrics
######


data = load_json(json_path, artists)
df = json_normalize(data)

lyrics_array = [] #this is ugly but I need those lyrics as single document in tf idf
lyrics = " "

for lyric in df.lyrics.values:
    allowed_chars = [" ","!", "?", "\n", "ä", "ö", "ü"] # allowed those chars + alphabet
    for i in range(97, 123):
        allowed_chars += chr(i)

    ignore_state = False
    filtered_lyric = ""
    for i in lyric:
        if i == "[":
            ignore_state = True
        if i == "]":
            ignore_state = False
        if not ignore_state:
            if i in allowed_chars:
                filtered_lyric += i
                
    lyrics_array.append(filtered_lyric)
    lyrics += " " + filtered_lyric

print('Corpus length: {}'.format(len(lyrics)))
lyrics

Corpus length: 195418


'  \nas war mit bstand die schlimmste oche\nie ich in meinem eben je hatte\nch wei nicht ob ich aufstehen soll\nnd ich hab keinn lan was ich mache\nenn irgendwie ist alles scheie\nch war noch nie so weit am oden\nenn ich wei mein ganzes eld ist weg\nnd meine rau hat mich betrogen\nah whatever!\n\n\nye bye ich fühl mich so frei frei\nch will nicht mehr heim\nenn mir ist scheiegal was morgen kommt\nch heb mein las und schrei\nye bye ich fühl mich so frei frei\nch will nicht mehr heim\nenn mir ist scheiegal was morgen kommt\nch heb mein las und schrei \n\n\nn den piegel ann ich brauche nichts auer dich\nir beide packen das ich hab dich du hast mich\nnd hat irgendjemand von uns beiden sich mal verpisst\nar das sicher wegen eld oder irgendeiner hick\netzt bin ich frei frei und steh wieder auf\nch bin allein und schrei s geht wieder raus\npring in die ikes fly und geh wie der aus\nn eitlupe in den lub es hat ewig gebraucht\noch es ist amstagnacht ich werd langsam wach\nnd bezahl die erste un

In [29]:
sentences = []
next_chars = []
    
for i in range(0, len(lyrics) - maxlen, step): # iterates by step size
    sentences.append(lyrics[i: i + maxlen]) # get maxlen amount of charachters
    next_chars.append(lyrics[i + maxlen])
    
print('Number of sequences:', len(sentences))

chars = sorted(list(set(lyrics))) # list of unique characters
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) # maps char with index

Number of sequences: 68719
Unique characters: 33


In [68]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '?': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'h': 11,
 'i': 12,
 'j': 13,
 'k': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'q': 20,
 'r': 21,
 's': 22,
 't': 23,
 'u': 24,
 'v': 25,
 'w': 26,
 'x': 27,
 'y': 28,
 'z': 29,
 'ä': 30,
 'ö': 31,
 'ü': 32}

In [69]:
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars))) # (sentences)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1    # one hot encoding
    y[i, char_indices[next_chars[i]]] = 1  # one hot encoding

Vectorization...


## Build Model

In [91]:
model = Sequential()
model.add(GRU(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [83]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 128)               62208     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 33)                4257      
Total params: 66,465
Trainable params: 66,465
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [92]:
model.fit(x, y, batch_size=128, epochs=1)

Epoch 1/1
68719/68719 [==============================] - 35s 513us/step - loss: 2.0848


In [11]:
model.save('./model_60epochs_luk.h5')

In [95]:
temperature = 0.3

start_index = random.randint(0, len(lyrics) - maxlen - 1)
generated_text = lyrics[start_index: start_index + maxlen]
print(generated_text)
#print('\n___________________\n')
for i in range(1500):
    sampled = np.zeros((1, maxlen, len(chars)))
            
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
                      
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char + "")

jeden tag allein und träumst wie schön es wär
gehst ins bett
 den ster st die meine nicht der stend den sten diesen wenn den sten wie der keine nicht den so der stend den schein nicht denn den scheit den den schein den sten wenn den sten ster den ster die auf den sten war der sten mich du scheit es ist dem zu den schein
denn geht den schei meine dann den es denn den compute
doch ben ich habe mach denn den scheit auf den stand ich bin ich habe wie der stert den ster die zu die den schein denn den schein ne ein auf den sties ist der nicht ster denn ich habe jetzt den schein den ster die ster st ich bin ich bin die zu das schein den schein den sten denn den steit auf den ne den stend ich haben ein alles den ster das ist den den ster den zu nicht den stein den den stend die wenn die denn der stelt denn den trauch die den ster ist den schein
und den ster schein auf du die stend den schein den stelt den trauss denn den sten den ster den scheit ein aus denn den scheie steht mich die scheie is

In [19]:
###
#  Rechtschreibung
###

from spellchecker import SpellChecker

spell = SpellChecker(language="de")

# find those words that may be misspelled
gen_words = generated_text.split(" ") #insert generated text

misspelled = spell.unknown(['Haus', 'ich', 'mich', 'schein']) 
number_of_misspelled_words= len(misspelled)

In [153]:
###
#  TF-IDF
###


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# calculate tf idf cos sim

lyrics_array.append("my generated text")

tfidf = v.fit_transform(lyrics_array)

cosine_similarities = linear_kernel(tfidf[1:], tfidf)

number_top_related_lyrics = 5
related_lyrics_idxs = cosine_similarities.argsort()[:-number_top_related_lyrics:-1] # extract top n related lyrics

top_counter = 1

for i in related_lyrics_idxs:
    related_artist = df.iloc[i].artist
    related_title = df.iloc[i].title
    print("#" + top_counter + " related title: "  + related_artist + " - " + related_title)
    top_counter += 1

related_lyrics_idxs

array([[  0,  71,  70,  69,  68,  66,  65,  64,  63,  61,  72,  60,  58,
         57,  56,  55,  53,  52,  51,  50,  49,  59,  48,  73,  75,  98,
         97,  96,  95,  93,  91,  90,  88,  87,  74,  86,  84,  83,  82,
         81,  80,  79,  78,  77,  76,  85,  47,  54,  45,  23,  22,  21,
         20,  17,  16,  15,  14,  24,  12,  10,   9,   8,   6,   5,   3,
          2,   1,  11,  25,  18,  34,  40,  27,  42,  39,  38,  37,  36,
         35,  33,  41,  43,  19,  62,  32,  46,  28,  29,   7,  92,  13,
         26,  30,  44,  89,   4,  31,  67,  94, 107,  99, 100, 101, 102,
        103, 104, 105, 106, 108],
       [  0,  71,  70,  69,  68,  66,  65,  64,  63,  61,  72,  60,  58,
         57,  56,  55,  53,  52,  51,  50,  49,  59,  48,  73,  75,  98,
         97,  96,  95,  93,  91,  90,  88,  87,  74,  86,  84,  83,  82,
         81,  80,  79,  78,  77,  76,  85,  47,  54,  45,  23,  22,  21,
         20,  17,  16,  15,  14,  24,  12,  10,   9,   8,   6,   5,   3,
          2,   1,

Number of misspelled words,0
Earth,6371
Moon,1737
Mars,3390
